In [2]:
import qiskit
import numpy as np
from qiskit import *
from math import pi

In [3]:
class CHSH_game:
    def __init__(self, rounds, eavesdrop=False):
        self.rounds = rounds
        self.eavesdrop = eavesdrop
        self.choices = [None, None]
        self.basis = [[0, pi/8, pi/4], [0, pi/8, -pi/8], [0, 0, 0]]
        self.game = [None]
        self.output = [[], []]
        if eavesdrop: 
            self.choices.append(None)
            self.output.append([])
        
    def create_round(self):
        alice = QuantumRegister(1, 'alice')
        bob = QuantumRegister(1, 'bob')
        m_a = ClassicalRegister(1, 'm_a')
        m_b = ClassicalRegister(1, 'm_b')
        if not self.eavesdrop:
            qc = QuantumCircuit(alice, bob, m_a, m_b)
        else:
            eve = QuantumRegister(1, 'eve')
            m_e = ClassicalRegister(1, 'm_e')
            qc = QuantumCircuit(alice, bob, eve, m_a, m_b, m_e)
        return qc
    
    def create_epr(self, qc):
        epr = QuantumCircuit(2)
        epr.h(0)
        epr.cx(0, 1)
        qc.compose(epr, qubits=[0, 1], inplace=True)
        
    def create_eavesdropper(self, qc):
        entanglement = QuantumCircuit(2)
        entanglement.cx(0, 1)
        qc.compose(entanglement, qubits=[1, 2], inplace=True)
        
    def create_entanglement(self):
        pass
    
    def gen_random_choice(self, length):
        from os import urandom
        choice = [int.from_bytes(urandom(1), 'big') % 3 for _ in range(length)]
        return choice
    
    def measure(self, player): # player = 0 / 1: Alice / Bob
        if player != 2:
            choice = self.gen_random_choice(self.rounds)
        else:
            choice = [0] * self.rounds # Eve should choose a strategy
        self.choices[player] = choice
        for i, c in enumerate(choice):
            basis = self.basis[player][c]
            self.game[i].ry(basis, player)
            self.game[i].measure(player, player)
            
    def retrieve_output(self):
        qasm_simulator = Aer.get_backend('qasm_simulator')
        for qc in self.game:
            result = execute(qc, qasm_simulator, shots=1, memory=True).result()
            memory = result.get_memory()[0].split()
            measured_bit_0 = int(memory[0])
            measured_bit_1 = int(memory[1])
            self.output[0].append(measured_bit_0)
            self.output[1].append(measured_bit_1)
            
    def evaluate(self):
        # TODO: CHSH inequality
        pass
    
    def init(self):
        self.game = [self.create_round() for _ in range(self.rounds)]
        for qc in self.game:
            self.create_epr(qc)
            if self.eavesdrop:
                self.create_eavesdropper(qc)
            
    def start(self):
        self.init()
        self.measure(0) # Alice measure
        self.measure(1) # Bob measure
        if self.eavesdrop: self.measure(2) # Eve measure
        self.retrieve_output()
        self.evaluate()

In [4]:
rounds = 1000
game = CHSH_game(rounds, True)
game.start()

In [5]:
game.game[0].draw()

┌───┐     ┌───────┐┌─┐                  
alice: ┤ H ├──■──┤ Ry(0) ├┤M├──────────────────
       └───┘┌─┴─┐└───────┘└╥┘┌──────────┐┌─┐   
  bob: ─────┤ X ├────■─────╫─┤ Ry(-π/8) ├┤M├───
            └───┘  ┌─┴─┐   ║ └┬───────┬─┘└╥┘┌─┐
  eve: ────────────┤ X ├───╫──┤ Ry(0) ├───╫─┤M├
                   └───┘   ║  └───────┘   ║ └╥┘
m_a: 1/════════════════════╩══════════════╬══╬═
                           0              ║  ║ 
                                          ║  ║ 
m_b: 1/═══════════════════════════════════╩══╬═
                                          0  ║ 
                                             ║ 
m_e: 1/══════════════════════════════════════╩═
                                             0